# Computação Gráfica 2025/2026

## Aula TP4 - Projecções em 3D com matplotlib e numpy - Projecções de câmara e perspectivas

### Um tutorial gráfico por André Falcão


Nesta aula vamos fazer projecções para a câmara e aprender a usar perspectivas, mas para isso temos que entender como projectamos qualquer objecto para um outro referencial. E começaremos com as projecções de pontos em Eixos


# 1.Projecções de pontos em eixos

A projecção de um ponto `p` ($p_x, p_y, p_z$)sobre um determinado eixo (`x,y,z`), consegue fazer-se multiplicando o vector do eixo devidamente normalizado por esse ponto p. Vamos ver um exemplo primeiro em 2D e depois em 3D


###  1.1 Projecção sobre um eixo em 2D

Primeiro vamos definir um eixo (e.g. (2,1) e para ser um vector ortonormado, normalizamo-o

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

axis = np.array([2,1], dtype=float)
axis = axis / np.linalg.norm(axis)
np.set_printoptions(precision=3)
print("Eixo normalizado:", axis)

Agora definimos um ponto qualquer em 2D (3,3) e para o projectar  temos que ter duas operações: Primeiro fazer o **produto interno** do eixo definido com esse ponto e isso consegue-se com o produto interno

$$ s = e^T.p $$

em que `e` é o eixo normalizado transposto para ser uma matriz linha, e `p` é o ponto que queremos projectar. Esta operação diz-nos 'quanto do vector do eixo'  é necessário para representar o ponto `p`. A segunda fase, para obtermos as coordenads reais do ponto projectado, temos que multiplicar pelo vector do eixo normalizado

$$ proj = s.e $$



In [ ]:
p = np.array([3, 3], dtype=float)

c = np.dot(axis, p) #1
p_proj = c * axis   #2
print("'Quantidade de p' projectada no eixo (2,1) ", c)
print("Coordenadas do ponto projectado no eixo (2,1):", p_proj)


Agora  é só representá-lo num eixo usando o nosso conhecido matplotlib

In [ ]:

# primeiro desenhamos o eixo (vindo da origem)
line_xn = np.linspace(0, axis[0], 100)
line_yn = (axis[1]/axis[0]) * line_xn

line_x = np.linspace(-1, 4, 100)
line_y = (axis[1]/axis[0]) * line_x
plt.plot(line_x, line_y, color="black", linewidth=0.5, label="Recta do Eixo (2,1)")
plt.plot(line_xn, line_yn, color="blue", linewidth=3, label="Eixo (2,1) normalizado")

plt.scatter(p[0], p[1], color="red", s=60, label="Ponto p")

plt.scatter(p_proj[0], p_proj[1], color="blue", s=60, label="Projecção de p")
plt.plot([p[0], p_proj[0]], [p[1], p_proj[1]], "k--")

plt.xlim=(-1,4)
plt.ylim=(-1,4)
plt.axhline(0, color='gray', linewidth=0.5)
plt.axvline(0, color='gray', linewidth=0.5)
plt.aspect= "equal"
plt.legend()
plt.title("Projecção do ponto p no eixo (2,1)")

plt.show()


#### Exercícios 1

1. Crie uma função `projecta_ponto_2d()`que aceite o eixo e ponoto e os represente
2. Usando a função acima, altere o ponto para projectar para `(2,0)`,`(2,0)`, e  `(1,-2)`
    1. Avalie e comente os valores do produto interno e a projecção
    2. Represente pontos e projecções usando o código acima

## 1.2. Projecção de um ponto em 3D

Obviamente que a transformação de um ponto em 2D funciona também em 3D e o princípio é exactamente o mesmo. Vamos usar a mesma biblioteca do matplotlib que vimos na última aula

Usaremos o eixo (1,2,1) e o ponto (2,1,5)

In [ ]:
# eixo e normalização
axis_init = np.array([1,2,1], dtype=float)
axis_norm = axis_init / np.linalg.norm(axis_init)

np.set_printoptions(precision=3)
print("Eixo normalizado: ", axis_norm)

In [ ]:

p = np.array([2,1,5], dtype=float)

#Projecçao do ponto sobre o vector

c = np.dot(axis_norm, p) #1a parte - avalia a "quantidade" da projecção
p_proj = c * axis_norm   #2a parte - multiplica a quantidade pelo valor do eixo

print("'Quantidade de p' projectada no eixo (1,2,1) normalizado", c)
print("Coordenadas do ponto projectado no eixo (1,2,1)", p_proj)

Agora basta desenhar o resultado com o `mplot3d` que também já conhecemos (note-se que o sistema de eixos não é o visto na aula passada, é o por omissão do matplotlib)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(7,6))
ax = fig.add_subplot(111, projection='3d')

ax.quiver(0,0,0, axis_norm[0], axis_norm[1], axis_norm[2], color="b", linewidth=3, label="Eixo(1,2,1) normalizado")
ax.quiver(0,0,0, axis_init[0], axis_init[1], axis_init[2], color="k", length=3, arrow_length_ratio=.1, linewidth=1, label="Eixo(1,2,1)")

ax.scatter(p[0], p[1], p[2], color="red", s=60, label="Ponto p")
ax.scatter(p_proj[0], p_proj[1], p_proj[2], color="blue", s=60, label="Projecção de p")
ax.plot([p[0], p_proj[0]], [p[1], p_proj[1]], [p[2], p_proj[2]], "k--")

ax.set_xlim(0,5)
ax.set_ylim(0,5)
ax.set_zlim(0,5)
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.legend()
ax.set_title("Projecção do ponto P Num eixo arbitrário (1,2,1)")

plt.show()


# 2. Projecções para o espaço da câmara


## 2.1 Interlúdio: O produto vectorial ou escalar


O **produto vectorial** (ou externo) de dois vetores em $\mathbb{R}^3$ é uma operação cujo resultado é:
* **perpendicular** (ortogonal) a $\mathbf{a}$ e a $\mathbf{b}$, o seu comprimento é $\|\mathbf{a}\|\;\|\mathbf{b}\|\sin\theta$;
* proporcional à área do paralelogramo formado;
* A sua direcção segue a **regra da mão direita**.

[Esta é uma operação vectorial relacionada com o cálculo do determinante de uma matriz com a da sobreposição dos dois vectores, mas sai fora do âmbito desta aula a forma como ele é calculado] 

O vector resultante do produto externo de $a \times\ b$ é
$$
\mathbf{a} \times \mathbf{b} =
\begin{bmatrix}
a_y b_z - a_z b_y \\
a_z b_x - a_x b_z \\
a_x b_y - a_y b_x
\end{bmatrix}
$$

Podemos ver um exemplo gráfico, com o produto externo de $(1,2,0) \times (0,1,1)$. O numpy tem uma função `np.cross` que faz logo essa operação

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Orthogonal basis vectors
a = np.array([1, -1, 0], dtype=float)  # j
b = np.array([1, 1, 0], dtype=float)  # i

# Dot and cross
dot_product = np.dot(a, b)       #se der zero os vectores são ortogonais    
cross_vec = np.cross(a, b)           
print("Resultado do produto interno", dot_product) #  Se forem ortogonais deverá dar zero 
print("Resultado do produto externo", cross_vec) 

Vamos verificar se `b` é perpendicular a `a`. Se sim, a sua projecção deverá ser nula

In [ ]:

# Projecção de b em a
a_norm = a / np.linalg.norm(a)
proj_b_on_a = (np.dot(b, a_norm)) * a_norm  

proj_b_on_a

Agora vamos visualizar o resultado no quadro de referência que já conhecemos

In [ ]:
fig = plt.figure(figsize=(7,6))
ax = fig.add_subplot(111, projection='3d')

ax.quiver(0,0,0, a[0], a[1], a[2], linewidth=2, label="a=%s" % str(a) , color="b")
ax.quiver(0,0,0, b[0], b[1], b[2], linewidth=2, label="b=%s" % str(b), color ="g")
#ax.quiver(0,0,0, proj_b_on_a[0], proj_b_on_a[1], proj_b_on_a[2], linewidth=2, label="Proj b em a", color="k")
ax.quiver(0,0,0, cross_vec[0], cross_vec[1], cross_vec[2], linewidth=2, label="a × b", color="r")
ax.set_xlim(-2,2)
ax.set_ylim(-2,2)
ax.set_zlim(-2,2)
ax.set_xlabel("X"); ax.set_ylabel("Y"); ax.set_zlabel("Z")
ax.legend()
ax.set_title("a·b = %s,  a×b=%s" % (str(dot_product), str(cross_vec)))

plt.show()


#### Exercícios 2

1. Responda verificando visualmente, o que acontece se a e b não forem perpendiculares?
2. Faça a=(-1,-1,2) e b=(0,1,1). Represente graficamente o resultado
3. Faça a=(-1,0,2) e b=(1,0,-2). Represente graficamente o resultado. Discuta os resultados


## 2.2. Projectar em 3D na câmara virtual. 

Uma câmara virtual só difere do que vimos acima porque em vez de um eixo tem três. Assim o que iremos fazer é projectar todos os vértices da cena para os 3 eixos que teremos na "câmara". Para definir uma câmara precisamos sempre de 3 componentes (tudo em coordenadas do mundo): 

1. posição do "olho" (`eye`)
2. Posição do ponto para onde se está a olhar  (`center`)
3. Qual o vector que indica o que está para cima  (`up`)




Agora para definir o sistema de coordenadas da câmara, precisamos de uma base ortonormal $(\mathbf{s}, \mathbf{u}, -\mathbf{f})$ Esta base corresponde aos nossos 3 eixos ($x, y, z$). Para os construir as seguintes operações são necessárias

1. **Vector forward (que aponta para a frente)** $(\mathbf{f}$): direção de visão, obtida como o vetor normalizado  
  $$
  \mathbf{f} = \frac{\mathbf{center} - \mathbf{eye}}{\|\mathbf{center} - \mathbf{eye}\|}.
  $$

2. **Vector side (que aponta para o lado direito)** \(\mathbf{s}\): perpendicular a $(\mathbf{f}$) e ao vetor “up” dado. Garante a orientação de mão-direita, vindo também normalizado:  
  $$
  \mathbf{s} = \frac{\mathbf{f} \times \mathbf{up}}{\|\mathbf{f} \times \mathbf{up}\|}.
  $$

3. **Vector up (que aponta para cima)** $(\mathbf{u}$): recalculado como o **produto vetorial** que vimos acima. Assim este vector será, por definição, unitário, mas também, pelas propriedades do produto externo, perpendicular a `f` e a `s` e seguirá a orientação da regra da mão direita
  $$
  \mathbf{u} = \mathbf{s} \times \mathbf{f}.
  $$

Assim, $(\mathbf{s}, \mathbf{u}, -\mathbf{f})$ forma uma base ortonormal da câmara, escrita em coordenadas do mundo.




A matriz de visualização \(V\) resulta da composição de duas transformações:

1. **Translação \(T\)**: move o ponto de vista (eye) para a origem da câmara:  
   $$
   T =
   \begin{bmatrix}
   1 & 0 & 0 & -\text{eye}_x \\
   0 & 1 & 0 & -\text{eye}_y \\
   0 & 0 & 1 & -\text{eye}_z \\
   0 & 0 & 0 & 1
   \end{bmatrix}.
   $$

2. **Rotação \(R\)**: muda da base do mundo para a base da câmara, usando os vetores construídos acima:  
   $$
   R =
   \begin{bmatrix}
   s_x & s_y & s_z & 0 \\
   u_x & u_y & u_z & 0 \\
   -f_x & -f_y & -f_z & 0 \\
   0 & 0 & 0 & 1
   \end{bmatrix}.
   $$

Multiplicando, obtemos:  
$$
V = R \cdot T.
$$

ou seja

$$
V \;=\;
\begin{bmatrix}
s_x & s_y & s_z & -\,\mathbf{s}\!\cdot\!\mathbf{eye} \\
u_x & u_y & u_z & -\,\mathbf{u}\!\cdot\!\mathbf{eye} \\
-\,f_x & -\,f_y & -\,f_z & \ \ \mathbf{f}\!\cdot\!\mathbf{eye} \\
0 & 0 & 0 & 1
\end{bmatrix},
\quad\text{com}\quad
\mathbf{f}=\dfrac{\mathbf{center}-\mathbf{eye}}{\|\mathbf{center}-\mathbf{eye}\|},\;
\mathbf{s}=\dfrac{\mathbf{f}\times\mathbf{up}}{\|\mathbf{f}\times\mathbf{up}\|},\;
\mathbf{u}=\mathbf{s}\times\mathbf{f}.
$$

Esta matriz ($V$) converte qualquer ponto em coordenadas do mundo para o **espaço da câmara**, projetando o vetor $(p - eye)$ sobre os eixos $(\mathbf{s}, \mathbf{u}, -\mathbf{f})$. Isto é a essÊncia do que é a função `LookAt_Matrix` que constroi uma matriz que projecta todos os objectos em coordenadas do mundo para coordenadas da câmara


In [ ]:
def normalize(v):
    n = np.linalg.norm(v)
    return v / n

def lookAt(eye, center, up):
    eye = np.array(eye, dtype=float)
    center = np.array(center, dtype=float)
    up = np.array(up, dtype=float)

    f = normalize(center - eye)          # direcção de vista
    s = normalize(np.cross(f, up))       # Direção do que está para a direita
    u = np.cross(s, f)                   # vector u recalculado (ortogonal aos outros dois

    
    V = np.array([
        [ s[0],  s[1],  s[2], -np.dot(s, eye)],
        [ u[0],  u[1],  u[2], -np.dot(u, eye)],
        [-f[0], -f[1], -f[2],  np.dot(f, eye)],
        [ 0.0,   0.0,   0.0,   1.0]
    ], dtype=float)

    return V

Vamos agora ver com seria uma matriz de lookAt com 

`
eye = (-1, 1,1)
center = (0,0,0)
up= (0,1,0)
`

In [ ]:
eye = (-1, 1,1)
center = (0,0,0)
up= (0,1,0)

lookAt(eye, center, up)

E vamos aplicá-la a uma pirâmide quadrangular com as seguintes coordenadas. 

**Veja-se que estamos a definir os vértices e depois componos os polígonos com listas de vértices!**

In [ ]:
verts_world = np.array([
    [0.0, 0.0, 0.0],  # 0
    [1.0, 0.0, 0.0],  # 1
    [1.0, 0.0, -1.0],  # 2
    [0.0, 0.0, -1.0],  # 3
    [0.5, 2.0, -0.5],  # 4 (topo)
], dtype=float) 

# Polígonos como listas de índices (fechados no desenho)
# Base e quatro faces laterais (triângulos)
poligonos = [
    [0, 1, 2, 3, 0],  # base (wireframe)
    [0, 1, 4, 0],
    [1, 2, 4, 1],
    [2, 3, 4, 2],
    [3, 0, 4, 3]
]


verts_world

Agora vamos fazer uma projecção para coordenadas de câmara, usando os polígonos e a matriz construída, devolvendo novos polígonos com as novas coordenadas


In [ ]:
def projecta_camara(verts, poligonos, V):
    #Recebe vértices Nx3 em mundo, lista de polígonos (índices) e matriz de view (4x4).
    #Devolve lista de polígonos em coords de câmara (cada polígono é array Mx3).
    N = verts.shape[0]
    verts_h = np.hstack([verts, np.ones((N, 1), dtype=float)])   # Nx4
    cam_h = (V @ verts_h.T).T                                    # Nx4
    # como V é afim, w=1; se não for, dividir por w:
    w = cam_h[:, 3:4]
    w = np.where(np.abs(w) < 1e-12, 1.0, w)
    verts_cam = cam_h[:, :3] / w
    # Construir lista de polígonos (coords de câmara)
    polis_cam = [verts_cam[idx_list, :] for idx_list in poligonos]
    return polis_cam



E agora podemos desenhar tudo numa nova projecção. Note-se que agora estamos a desenhar um objecto em 3D usando o Matplotlib apenas. sem qualquer biblioteca 3D

In [ ]:
from matplotlib.patches import Polygon as MplPolygon

def desenha_projeccao(poligonos_cam, figsize=(6,6),
                      titulo="Projecção na câmara (x,y)", alpha=0.2):
    plt.figure(figsize=figsize)
    ax = plt.gca()

    for i, P in enumerate(poligonos_cam):
        x = P[:, 0]
        y = P[:, 1]
        pontos = np.c_[x, y]
        poly = MplPolygon(pontos, closed=True,
                          facecolor="blue",
                          edgecolor='k', alpha=alpha, linewidth=1.5)
        ax.add_patch(poly)
    ax.set_aspect('equal', adjustable='box')
    ax.set_xlabel("x (câmara)")
    ax.set_ylabel("y (câmara)")
    ax.set_title(titulo)
    ax.grid(True, linestyle='--', alpha=0.4)

    ax.set_xlim(-2, 2)
    ax.set_ylim(-1, 3)
    ax.set_aspect("equal")

    plt.show()


#esta função faz o mesmo mas representa o objecto em Wireframe - pode ser útil em determinados contextos
def desenha_projeccao_wf(poligonos_cam, figsize=(6,6), titulo="Projecção na câmara (x,y)"):
    plt.figure(figsize=figsize)
    ax = plt.gca()
    for P in poligonos_cam:
        x = P[:, 0]  # x_câmara
        y = P[:, 1]  # y_câmara
        ax.plot(x, y, linewidth=1, color="k")
    ax.set_aspect('equal', adjustable='box')
    ax.set_xlabel("x (câmara)")
    ax.set_ylabel("y (câmara)")
    ax.set_title(titulo)
    ax.grid(True, linestyle='--', alpha=0.4)

    ax.set_xlim(-2, 2)
    ax.set_ylim(-1, 3)
    ax.set_aspect("equal")
    plt.show()



Já temos então tudo! Podemos fazer as projecções para a câmara (sem qualquer efeito de perspectiva)

Assim o que faremos será
1. Definir parâmetros das câmara
2. criar matriz de visualização
3. Projectar polígonos para referencial da câmara
4. Desenhhar a projecção


In [ ]:

eye    = (0.0, 0.0,  1.0)
up     = (0.0, 1.0,  0.0)
center = (0,-1,0) 

V = lookAt(eye, center, up)

poligonos_cam = projecta_camara(verts_world, poligonos, V)
desenha_projeccao(poligonos_cam, titulo="Pirâmide – projecção x,y em coords de câmara")

Veja-se o que acontece quando colocamos a câmara em cima, directamente a olhar para baixo

In [ ]:
eye    = (0.0, 3.0,  0.0) #ALTO
up     = (0.0, 0.0,  1.0)
center = (0,1,0)

V = lookAt(eye, center, up)

poligonos_cam = projecta_camara(verts_world, poligonos, V)
poligonos_cam
desenha_projeccao(poligonos_cam, titulo="Pirâmide – projecção x,y em coords de câmara")

# 3. Aplicação de perspectivas


### Construção de perspectivas - Matriz de Projecção em Perspectiva



Como vimos na aula, a transformação de **câmara → clip space** é dada pela matriz $ P $:

$$
P =
\begin{bmatrix}
\frac{f}{a} & 0 & 0 & 0\\[4pt]
0 & f & 0 & 0\\[4pt]
0 & 0 & \frac{f_c + n}{n - f_c} & \frac{2 f_c n}{n - f_c}\\[4pt]
0 & 0 & -1 & 0
\end{bmatrix}
$$

onde:

- $ f = \dfrac{1}{\tan(\tfrac{\mathrm{fov}_y}{2})} $ é o fator de focalização vertical;  
- $ a $ é o **aspect ratio** (largura/altura do viewport);  
- $ n $ é a distância do **plano próximo (near)** (positivo, em frente da câmara);  
- $ f_c $ é a distância do **plano longe (far)**;  

Para um ponto da câmara $ p = (x, y, z, 1) $, o resultado é:

$$
p' = P \, p = (x', y', z', w')
$$

Finalmente e muito importante, a transformação para **Normalized Device Coordinates (NDC)** é feita pela *perspective divide*, que vai permitir que elementos mais distantes apareçam mais pequenos. Isto consegue-se porque, depois da transformação em coordenadas da câmara, temos os Zs para cada vértice. Mas até agora temos estado a usar apenas o z e o y da projecção ignorando o que sucede no eixo do z (e por isso é que temos compressão do ponto de vista). O perspective divide é feito sempre posteriormente e é feito simplesmente dividindo os valores das 3 coordenadas pelo valor de $w'$

$$
(x_{ndc}, y_{ndc}, z_{ndc}) = \frac{(x', y', z')}{w'}
$$

onde $ x_{ndc}, y_{ndc}, z_{ndc} \in [-1, 1] $.

A matriz Perspectiva pode ser constuída assim, trivialmente por aplicação da fórmula


In [ ]:


def perspectiva(fov_y_graus, aspect, near, far):
    #Matriz de projecção perspectiva (estilo OpenGL):
    fov_y = np.deg2rad(fov_y_graus)
    f = 1.0 / np.tan(fov_y * 0.5)
    n, fc = float(near), float(far)

    P = np.array([
        [f/aspect, 0.0, 0.0,   0.0],
        [0.0,      f,   0.0,   0.0],
        [0.0,      0.0, (fc+n)/(n-fc), (2.0*fc*n)/(n-fc)],
        [0.0,      0.0, -1.0,  0.0]
    ], dtype=float)
    return P


A função `transforma_perspectiva` recebe uma lista de polígonos em coordenadas da CÂMARA (cada P é Mx3: x,y,z), aplica a projecção perspectiva P (4x4), faz a divisão por w (*perspective divide*)
e devolve lista com polígonos em *coordenadas normalizadas do dispositivo* (NDC) (Mx3: x_ndc, y_ndc, z_ndc).

O argumento  'descartar_fora' serve para descartar vértices com w'<=0 (atrás da câmera) ou não finitos.

Vamos ainda necessitar de uma função nova de desenho cuja única e relevante alteração é definir os limites da imagem entre (-1 e 1) porque são coordenadas normalizadas

In [ ]:
def transforma_perspectiva(poligonos_cam, P, descartar_fora=True):
    polis_ndc = []
    for Pc in poligonos_cam:
        M = Pc.shape[0]
        ph = np.hstack([Pc, np.ones((M,1), dtype=float)])   # Mx4
        clip = (P @ ph.T).T                                 # Mx4
        w = clip[:, 3:4]

        #esta parte é para situações de projejcções numericamente instáveis ou para pontos atrás da câmara
        # evitar divisões inválidas
        mask = np.isfinite(w) & (np.abs(w) > 1e-12)
        mask = mask[:,0]

        # para remover pontos atrás (w<=0) ou não finitos
        if descartar_fora: mask = mask & (w[:,0] > 0)

        clip_ok = clip[mask]
        w_ok = clip_ok[:, 3:4]
        ndc = clip_ok[:, :3] / w_ok                         

        # se todos ficaram fora, devolve polígono vazio para manter alinhamento
        if ndc.size == 0:
            polis_ndc.append(np.zeros((0,3), dtype=float))
        else:
            polis_ndc.append(ndc)

    return polis_ndc



def desenha_projeccao_ndc(poligonos_cam, figsize=(6,6),
                      titulo="Projecção perspectiva (NDC)", alpha=0.1):
    plt.figure(figsize=figsize)
    ax = plt.gca()

    for i, P in enumerate(poligonos_cam):
        x = P[:, 0]
        y = P[:, 1]
        pontos = np.c_[x, y]
        poly = MplPolygon(pontos, closed=True,
                          facecolor="blue",
                          edgecolor='k', alpha=alpha, linewidth=1.5)
        ax.add_patch(poly)

    ax.set_aspect('equal', adjustable='box')
    ax.grid(True, linestyle='--')

    ax.set_xlim(-1, 1)  #<- AQUI está
    ax.set_ylim(-1, 1)  #<- AQUI está
    ax.set_xlabel("x (NDC)")
    ax.set_ylabel("y (NDC)")
    ax.set_title(titulo)

    plt.show()


Vamos agora definir um ponto de observação em coordenadas do mundo e representar a nossa pirâmide. Primeiro vamos definir a matriz de projecção perspectiva que transforma coordenadas do mundo em coordenadas

Neste exemplo vamos usar um FOV de 90 graus e far e near de 0.1 e 100.0 respectivamente

In [ ]:
eye    = (0.0, 1.0,  3.0)
up     = (0.0, 1.0,  0.0)
center = (1,0,0)
V = lookAt(eye, center, up)
# 1. - Transformar para coordenadas da câmara
poligonos_cam = projecta_camara(verts_world, poligonos, V)

# 2. - Cria matriz de projecção perspectiva
P = perspectiva(fov_y_graus=90.0, aspect=1.0, near=0.1, far=100.0)

print(P)

Agora aplicamos a transformação P e perspective divide para coordenadas NDC com a função definida acima. Assim teremos o resultado em coordenadas NDV

In [ ]:
poligonos_ndc = transforma_perspectiva(poligonos_cam, P, descartar_fora=True)


desenha_projeccao_ndc( poligonos_ndc, titulo="Pirâmide — projecção em perspectiva (NDC)")

#### Exercícios 3

1. Modifique cada uns dos parâmetros da câmara (eye, up e center), tente prever o que vai acontecer e só depois execute
    1. O que acontece com o fov?
    2. O que acontece com os planos near and far
    3. O que acontece com o `center, eye e up`?
2. Faça uma função `LookAtPersp` que receba os parâmetros da câmara e toda a informação necessária para compor uma perspectiva e devolva a matriz combinada de visualização e perspectiva. Explore como consegue integrar essa informação com o pipeline gráfico já dado para, dada uma cena no formato aqui mencionado
3. Veja com atenção o código abaixo, que representa uma cena complexa com 3 objectos (2 pirâmides e 1 cubo) e que usa um módulo (shape_utils) incluído nesta aula
   1. experimente modificar o  ponto de observação e veja o que acontece
   2. Usando o conhecimento que já tem das outras aulas, rode o cubo 45 graus sobre o eixo (-1,1,1), centrado no seu centroide, à pirâmide 1, rode-a sobre a origem no eixo dos yy (12 graus no sentido directo) e à pirâmide 2, escale-a (passando o centroide e para a origem primeiro) para 70% do seu tamannho em todos os eixos

**NOTA:** Para o último exercício é muito importante pensar primeiro na ordem das transformações!

In [ ]:

import shape_utils
shape_utils


pira1 = shape_utils.cria_piramide(apice=[0.5, 1.0, -0.5], centro_base=[0.5, 0.0, -0.5], lado=1.0)
pira2 = shape_utils.cria_piramide(apice=[2.0, .5,  -2.0], centro_base=[2.0, -1.0,  -2.0], lado=0.8)
cubo1 = shape_utils.cria_cubo(centro=[-1.5, -0.5, -1.5], lado=1.2)

cena = shape_utils.junta_objectos([pira1, pira2, cubo1])


##### As funcões abaixo podem ser combinadas numa única  (exerc. 2.)###
V = lookAt(eye=[0,2,2], center=[0,0,0], up=[0,1,0])
cena_cam = projecta_camara(cena["verts"], cena["polys"], V)
P = perspectiva(fov_y_graus=60, aspect=1.0, near=0.1, far=100.0)
######################################################################

poligonos_ndc = transforma_perspectiva(cena_cam, P)

desenha_projeccao_ndc(poligonos_ndc, titulo="Cena combinada — perspectiva (NDC)", alpha=0.2)